In [ ]:
# Import libraries
import numpy as np
from matplotlib import pyplot as plt
# import pysindy as ps
from Yukawa_SINDy import *
# from importlib import reload

In [ ]:
# Stuff used in Yukawa_SINDy.py
from pysindy.differentiation import FiniteDifference
from scipy.integrate import solve_ivp
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error # doesn't work, but works in the file 'aps_workshop.ipynb'. why??
from sklearn.linear_model import Lasso

# For reproducibility
np.random.seed(2738) # defined later in function

# Solver keywords
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12 # set relative tolerance
integrator_keywords['method'] = 'LSODA' # # Livermore Solver for Ordinary Differential Equations with Automatic Stiffness Adjustment
integrator_keywords['atol'] = 1e-12 # set absolute tolerance

In [ ]:
# ignore warnings
# stuff to hide warnings (doesn't work)
import warnings
from copy import copy
from contextlib import contextmanager
from scipy.linalg import LinAlgWarning
from sklearn.exceptions import ConvergenceWarning

@contextmanager
def ignore_specific_warnings():
    filters = copy(warnings.filters)
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    warnings.filterwarnings("ignore", category=LinAlgWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    yield
    warnings.filters = filters

# Yukawa equation of motion

## Demonstration of `Yukawa_simulation` class:

We have defined a python object class which will simulate this physical system. Instantiate an instance of `Yukawa_simulation` in order to generate data. Use the `.simulate()` method to simulate, the first argument is the duration of simulation in seconds. Use the `.plot()` method to view the 

In [ ]:
example_sim = Yukawa_simulation()
example_sim.simulate(3)#, 0.001, 1, 0.01) # all default values
example_sim.plot()

Check if the simulation has had noise added to it using the `is_noisy` attribute.

In [ ]:
example_sim.is_noisy

Add gaussian-distributed noise to the data by using the `.add_gaussian_noise()` method. Default noise standard deviation is $1\%$.

In [ ]:
example_sim.add_gaussian_noise()
example_sim.plot()

Change the noise level by first deleting any noise which was already added using the `.delete_noise()` method. 

In [ ]:
example_sim.delete_noise()
example_sim.plot()

Then, add different noise level by entering desired standard deviation as the first argument of the `.add_gaussian_noise()` method.

In [ ]:
example_sim.add_gaussian_noise(0.05)
example_sim.plot()

Check standard deviation of added noise with the `noise_level` attribute.

In [ ]:
example_sim.noise_level

Subsample the noisy (or clean) data using the `.subsample()` method. Randomly samples $10\%$ of data by default:

In [ ]:
example_sim.subsample()
example_sim.plot()

To restore the full data, use the `.restore_data()` method.

In [ ]:
example_sim.restore_data()
example_sim.plot()

Note: the program will not add noise to subsampled, clean data:

In [ ]:
example_sim.delete_noise()
example_sim.subsample()
example_sim.add_gaussian_noise()

All data is stored in the 

In [ ]:
# Deprecated code:

# # x is a two-column vector, first column (x[0]) is position, second column (x[1]) is velocity
# # Normalized Yukawa EOM, call all constants 1
# def Yukawa_EOM(t, x): return [x[1], ( 1/x[0] + 1/x[0]**2 ) * np.exp( -x[0] ) ]

# def simulate_Yukawa(duration, dt=0.001, x0=1, v0=0.01, integrator_keywords=integrator_keywords):
#     # Generate measurement data
#     t_train = np.arange(0, duration, dt)
#     t_train_span = (t_train[0], t_train[-1])

#     x0_train = [x0, v0]
#     x_clean = solve_ivp(Yukawa_EOM, t_train_span, x0_train, t_eval=t_train, **integrator_keywords).y.T
#     return t_train, x_clean

# def plot_position_and_velocity(t_train, x_clean):
#     plt.xlabel("time (a. u.)")
#     plt.plot(t_train, x_clean[:,0], label='position')
#     plt.plot(t_train, x_clean[:,1], 'r', label='velocity')
#     plt.legend()
#     plt.show()

## Fitting a SINDy model to clean data

We will begin looking at a dusty-plasma-relevant system and trying to use SINDy to deduce the equations of motion directly from simulated data. We will begin by using clean data with no noise added, followed by adding Gaussian-distributed noise to the data to test the robustness of the method in the specific case of two particles which interact purely with the Yukawa potential.

We will simulate a two-particle system which evolves according to the Yukawa equations of motion,

$\frac{dx}{dt} = v$

$\frac{dv}{dt} = \left( \frac{1}{x} + \frac{1}{x^2} \right) e^{-x} $,

where $x$ is the interparticle separation. We will begin by generating some data:

In [ ]:
sim1 = Yukawa_simulation()
sim1.simulate(3)
sim1.plot()

Try with sort of a stupid library of coefs that I know will work. Will try with:

$\begin{pmatrix}
    \vert & \vert & \vert & \vert & & \vert & \vert \\
    x & v & \frac{1}{x}e^{-x} & \frac{1}{v}e^{-v} & \dots & \frac{1}{x^4}e^{-x} & \frac{1}{v^4}e^{-v} \\
    \vert & \vert & \vert & \vert & & \vert & \vert
\end{pmatrix}$

In [ ]:
# Fit SINDy model with custom library:
clean_model = fit_Yukawa_model(sim1,hparam=0)
clean_model.print()

In [ ]:
clean_model.get_feature_names()

Let's find exactly when, as we scan through threshold values, the SINDy model converges to the correct model.

In [ ]:
explore_thresholds(sim1, 0.0, 0.3, 0.01, plot=True, verbose=True)

Fits perfectly! Now, let's look at the coefficient matrix and complexity as a sanity check.

In [ ]:
clean_model.coefficients().T

In [ ]:
clean_model.complexity

In [ ]:
plot_coefs(clean_model)

Score does not make sense. Need to check on this later.

In [ ]:
clean_model.score(sim1.x)

Can also plot the SINDy model predicted derivatives and compare them to the derivatives calculated from data using finite difference.

In [ ]:
plot_derivatives(sim1,clean_model)

In [ ]:
explore_thresholds(sim1, 0.0, 0.3, 0.01, plot=True,verbose=False)

In [ ]:
test_model = fit_Yukawa_model(sim1,hparam=0.01)
plot_coefs(test_model,hparam=0.01,)

### Generating a Pareto Plot

To cross-validate our model, we can look at another trajectory (different initial conditions) and check if that model does well. To begin, we simulate with initial position of 1, initial velocity of -1.

In [ ]:
test_sim = Yukawa_simulation()
test_sim.simulate(3, dt=0.001, x0=1, v0=-1)
test_sim.plot()

In [ ]:
plot_pareto(sim1.x,sim1.t,np.arange(0.0,0.1,0.01),test_sim.x,test_sim.t)

### Other optimizers

#### LASSO

In [ ]:
model_lasso = fit_Yukawa_model(sim1,opt_str='lasso',hparam=0.5)
model_lasso.print()

plot_derivatives(sim1,model_lasso)

#### SR3

In [ ]:
model_sr3 = fit_Yukawa_model(sim1,opt_str='sr3',hparam=0.1)
model_sr3.print()

plot_derivatives(sim1,model_sr3)

In [ ]:
del sim1

## Noise Robustness

Fitting a SINDy model to noisy data

We now generate noisy data with the `add_gaussian_noise` attribute of the `Yukawa_simulation` class. We will start a new instance of the class to avoid conflict with the cells above. We can plot the noisy data against the clean data to have a visual representation of it.

### `std_dev=0.001`

In [ ]:
sim0001 = Yukawa_simulation()
sim0001.simulate(3) # simulate for 3 seconds
sim0001.add_gaussian_noise(0.001)
sim0001.plot()

Check noise level:

In [ ]:
sim0001.noise_level

Model with noisy data:

In [ ]:
# fit model and explore different thresholds
hparam_space = (0.0,0.6,0.001)
explore_thresholds(sim0001, *hparam_space, plot=True, verbose=False)

In [ ]:
for i in np.arange(0.1, 0.201, 0.001,dtype=np.longdouble):
    print('threshold =',np.round(i,3))
    fit_Yukawa_model(sim0001,opt_str='stlsq',hparam=i).print()
    print('------------------------------')
    print('\n')


### `std_dev=0.003`

In [ ]:
sim0030 = Yukawa_simulation()
sim0030.simulate(3) # simulate for 3 seconds
sim0030.add_gaussian_noise(0.003)
sim0030.plot()

In [ ]:
latex_feature_names = [
    r'$r$',
    r'$v$',
    r'$\frac{e^{-r}}{r}$',
    r'$\frac{e^{-v}}{v}$',
    r'$\frac{e^{-r}}{r^2}$',
    r'$\frac{e^{-v}}{v^2}$',
    r'$\frac{e^{-r}}{r^3}$',
    r'$\frac{e^{-v}}{v^3}$',
    r'$\frac{e^{-r}}{r^4}$',
    r'$\frac{e^{-v}}{v^4}$'
]
hparam_space = (0.0,1.0,0.01)
explore_thresholds(sim0030, *hparam_space, plot=True, verbose=False, feature_names=latex_feature_names)

something very interesting happening around `hparam=0.3`. It finds the correct *leading terms*, but then gets rid of them and goes to other terms. Let's investigate around this point more.

In [ ]:
hparam_space = (0.2,0.6,0.02)
explore_thresholds(sim0030, *hparam_space, plot=True, verbose=False)

In [ ]:
test_model = fit_Yukawa_model(sim0030, hparam=0.32)
test_model.print()
print()
test_model2 = fit_Yukawa_model(sim0030, hparam=0.42)
test_model2.print()

Let's compare a model within the local minimum to a model outside of the local minimum with the same number of terms.

In [ ]:
model0030 = fit_Yukawa_model(sim0030, hparam=0.3)
model0030.print()
print('\n')
# plot_coefs(model0030,hparam=0.3)
model0030 = fit_Yukawa_model(sim0030, hparam=0.8)
model0030.print()
# plot_coefs(model0030,hparam=0.8)

In [ ]:
for threshold in np.arange(0.2,0.401,0.001):
    print('------------------------------')
    print('\n')
    print('threshold =',np.round(threshold,3))
    fit_Yukawa_model(sim0030,hparam=threshold).print()

So we see that we have *very* different models. The `hparam=0.3` model correctly identifies the leading terms in the model, and while the `hparam=0.8` has the same number of terms it does not identify the correct leading terms. In other words, even though they have the same complexity, it's almost as if the STLSQ optimizer is converging on the wrong model over the more correct one with higher hyperparameter. Therefore, I believe this is a nonconvex problem--in the optimization, there exist local minima which may or may not identify the true model.

### `std_dev=0.01`

In [ ]:
sim0100 = Yukawa_simulation()
sim0100.simulate(3)
sim0100.add_gaussian_noise(0.01)
sim0100.plot()

In [ ]:
hparam_space = (0.0,1,0.01)
explore_thresholds(sim0100, *hparam_space, plot=True, verbose=False)

Seems to be a local minimum at around `hparam=0.32`. Investigating further:

In [ ]:
hparam_space = (0.29,0.37,0.001)
explore_thresholds(sim0100, *hparam_space, plot=True, verbose=False)

Looking at a particular model in this local min,

In [ ]:
model0010 = fit_Yukawa_model(sim0100, hparam=0.32)
model0010.print()
# plot_coefs(model0010,hparam=0.32)

These models don't seem special in any particular way, other than the fact that they form a local min on the # vs threshold plot. 

There is also an interesting point around 0.53 where the correct terms appear in the $\dot{v}$ equation. Let's look at this some:

In [ ]:
hparam_space = (0.5,0.6,0.001)
explore_thresholds(sim0100, *hparam_space, plot=True, verbose=False)

A particular model from this region:

In [ ]:
model0010 = fit_Yukawa_model(sim0100, hparam=0.54)
model0010.print()

While the $\dot{v}$ equation has the correct terms here, the $\dot{x}$ equation is *completely* incorrect.

Looking into the threshold values in the range of around 0.60 to 0.80, where the algorithm seems to finally converge:

In [ ]:
# fine-tuning for finding transition point
hparam_space = (0.50,0.80,0.003)
explore_thresholds(sim0100, *hparam_space, plot=True, verbose=False)

Model with correct number of terms:

In [ ]:
model0010 = fit_Yukawa_model(sim0100, hparam=0.63)
model0010.print()

Final model:

In [ ]:
model0010 = fit_Yukawa_model(sim0100, hparam=0.75)
model0010.print()

So, the model finally converges to have the correct *leading terms*, with around `hparam=0.75`. However, the coefficient on the first order rational term is closer to 2 when it should be closer to 1.

Interestingly, the models which have the same complexity as the true model (number of terms = 3) do not capture the correct dynamics. Between `hparam=0.70` and `hparam=0.75`, we move from 3 terms to 2, and see an improvement in the model--it finds the correct leading terms at least qualitatively. Investigating this transition further,

In [ ]:
# fine-tuning for finding transition point
hparam_space = (0.70,0.75,0.001)
explore_thresholds(sim0100, *hparam_space, plot=True, verbose=False)

Aha! moving from 0.70 to 0.71 chops two terms from the $\dot{x}$ equation, and from 0.73 to 0.74 moves from identifying the wrong leading term to identifying the correct leading term in the $\dot{v}$ equation.

### `std_dev=0.03`

In [ ]:
sim0300 = Yukawa_simulation()
sim0300.simulate(3)
sim0300.add_gaussian_noise(0.03)
sim0300.plot()

In [ ]:
hparam_space = (0.0,2.0,0.01)
explore_thresholds(sim0300, *hparam_space, plot=True, verbose=True)

### `std_dev=0.05`

In [ ]:
sim0500 = Yukawa_simulation()
sim0500.simulate(3)
sim0500.add_gaussian_noise(0.05)
sim0500.plot()

In [ ]:
hparam_space = (0.0,15.1,0.1)
explore_thresholds(sim0500, *hparam_space, plot=True, verbose=True)

### `std_dev=0.1`

In [ ]:
sim1000 = Yukawa_simulation()
sim1000.simulate(3)
sim1000.add_gaussian_noise(noise_level=0.1)
sim1000.plot()

I forsee that this one will greatly benefit from sub-sampling the data, because it is so noisy.

Also, it doesn't seem like this amount of noise is very physical. Do we all mean the same thing when we say "ten percent noise?"

In [ ]:
hparam_space = (0.0,16,0.25)
explore_thresholds(sim1000, *hparam_space, plot=True, verbose=False)

In [ ]:
hparam_space = (0.0,16,2)
complexity, hparams = explore_thresholds(sim1000, *hparam_space, plot=True, verbose=False)
plot_complexity(complexity, hparams, *hparam_space)

In [ ]:
for i, threshold in enumerate(np.arange(12.0,14.1,0.1)):
    if i != 0:
        print('------------------------------')
        print('\n')
    print('threshold =',np.round(threshold,3))
    fit_Yukawa_model(sim1000,hparam=threshold).print()

### Cross-validate using different trajectory

In [ ]:
test_sim = Yukawa_simulation()
test_sim.simulate(3, dt=0.001, x0=1, v0=-1)
test_sim.plot()

In [ ]:
sim0001.plot()

In [ ]:
plot_pareto(sim0001, test_sim, np.arange(0.0,1.1,0.1))

### Subsampling

Let's try subsampling the noisy data with 10% noise added to see if the SINDy models improve.

In [ ]:
sim1000.restore_data()

In [ ]:
sim1000.subsample(0.1)
sim1000.plot()

In [ ]:
model1000 = fit_Yukawa_model(sim1000, hparam=30)
# plot_coefs(model1000)

In [ ]:
hparam_space = hparam_space = (0.0,32,4)
complexity_sub, hparams_sub = explore_thresholds(sim1000, *hparam_space, plot=False, verbose=False)
# plot_complexity(complexity, hparams, *hparam_space)

In [ ]:
sim1000.restore_data()
complexity_full, hparams_full = explore_thresholds(sim1000, *hparam_space, plot=False, verbose=False)
# plot_complexity(complexity, hparams, *hparam_space)

In [ ]:
plt.plot(hparams_sub,complexity_sub,'.', label='subsampled')
plt.plot(hparams_full,complexity_full,'.', label='full')
plt.legend()
plt.xlabel("Threshold Value")
plt.ylabel("Number of terms")
plt.show()

In [ ]:
model1000 = fit_Yukawa_model(sim1000, hparam=32)
plot_coefs(model1000)

### Noise Figure

In [ ]:
duration = 3
dt = 1e-3
num_steps = int(duration/dt)

noises = [1e-3, 5e-2, 1e-1]
all_noisy_data = np.empty((0,num_steps))
for noise in noises:
    noisy_sim = Yukawa_simulation()
    noisy_sim.simulate(3)
    noisy_sim.add_gaussian_noise(noise_level=noise)
    all_noisy_data = np.vstack((all_noisy_data, noisy_sim.x.T))

# print(all_noisy_data.shape)
fig, ax = plt.subplots(1,1)
styles = ['k','c','b']
for i in range(all_noisy_data.shape[0]-1,0,-2):
    ax.plot(noisy_sim.t,all_noisy_data[i],styles[i//2],label=str(noises[i//2]),linewidth=2)
    ax.plot(noisy_sim.t,all_noisy_data[i-1],styles[i//2],linewidth=2)
    ax.legend()
    print(i)
    # break
ax.set_xlabel("Time (s)")
ax.set_ylabel("Position and Velocity (arb. u.)")
ax.set_title("Data at different noise levels")


In [ ]:
noises[::2]

In [ ]:
noisy_sim = Yukawa_simulation()
noisy_sim.simulate(3)
noisy_sim.add_gaussian_noise(noise_level=noise)
# all_noisy_data = np.vstack((all_noisy_data, noisy_sim.x[0]))
noisy_sim.x.T[0]